# Prediction of Active Enhancers with FFNN

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from itertools import product
from typing import List
from tqdm.auto import tqdm

## Models

### Decision Trees

### Random forests

### Extra Trees

### Support Vector Machine (SVM)

### Perceptron

### Multi-Layer Perceptron (MLP)

### Feed-Forward Neural Network (FFNN)

## Meta-models

### Grid search

### Random search

### Tree of Parzen

### Bayesian optimization with gaussian processes

## Kernel space

### Gaussian kernels

### Nystroem Kernels (approximated Gaussian kernels)

## The experimental setup

## Retrieving the data

In [2]:
from epigenomic_dataset import load_epigenomes

window_size = 200

X, y = load_epigenomes(
    cell_line = "GM12878",
    dataset = "fantom",
    regions = "promoters",
    window_size = window_size
)

X = X.droplevel(1, axis=1) 

## Preparing the holdouts

## Training the models

## Collecting the results

## Conclusions